# Conversión de Máscara Binaria a formato COCO JSON

## Configuración Previa

In [ ]:
import glob
import json
import os
import cv2
from tqdm import tqdm

In [ ]:
# Conectar con Google Drive. Recomendado para la ejecución en Google Colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path_dir = "/content/drive/MyDrive/TFM/MSLesSeg-Dataset" # modificar la ruta a la que contiene las imágenes

In [ ]:
# Establecemos las rutas de las imágenes, máscaras y los archivos de las futuras anotaciones
train_mask_path = f"{path_dir}/output_train/P1/T1/P1_T1_MASK.nii"
train_json_path = f"{path_dir}/annotations_train/P1/T1/train.json"

val_mask_path = f"{path_dir}/output_test/P54/T54/P54_T54_MASK.nii"
val_json_path = f"{path_dir}/annotations_test/P54/T54/test.json"

## Conversión

In [ ]:
# Definimos el id de la clase a detectar en nuestro proyecto
category_ids = {
    "Lesion": 1,
}

MASK_EXT = 'png'
ORIGINAL_EXT = 'png'
image_id = 0
annotation_id = 0

def images_annotations_info(maskpath):

    global image_id, annotation_id
    annotations = []
    images = []

    # Se itera para cada categoría y sus correspondientes máscaras
    for category in category_ids.keys():
        mask_files = glob.glob(os.path.join(maskpath, f'*.{MASK_EXT}'))
        print(mask_files)
        for mask_image in tqdm(mask_files, desc=f'Procesando {category}', unit='imagen'):
            original_file_name = f'{os.path.basename(mask_image).split(".")[0]}.{ORIGINAL_EXT}'
            mask_image_open = cv2.imread(mask_image)

            # Guardamos las dimensiones de las imágenes
            height, width, _ = mask_image_open.shape

            # Se crea o modifica el archivo con la anotación de la imagen
            if original_file_name not in map(lambda img: img['file_name'], images):
                image = {
                    "id": image_id + 1,
                    "width": width,
                    "height": height,
                    "file_name": original_file_name,
                }
                images.append(image)
                image_id += 1
            else:
                image = [element for element in images if element['file_name'] == original_file_name][0]

            # Se encuentran los polígonos que representan las máscaras mediante funciones de OpenCV
            gray = cv2.cvtColor(mask_image_open, cv2.COLOR_BGR2GRAY)
            _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
            contours = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)[0]

            # Se registran las anotaciones para cada polígono
            for contour in contours:
                bbox = cv2.boundingRect(contour)
                area = cv2.contourArea(contour)
                segmentation = contour.flatten().tolist()

                annotation = {
                    "iscrowd": 0,
                    "id": annotation_id,
                    "image_id": image['id'],
                    "category_id": category_ids[category],
                    "bbox": bbox,
                    "area": area,
                    "segmentation": [segmentation],
                }

                # Si el área es mayor que 0 se añade como anotación
                if area > 0:
                    annotations.append(annotation)
                    annotation_id += 1

    return images, annotations, annotation_id


def process_masks(mask_path, dest_json):
    global image_id, annotation_id
    image_id = 0
    annotation_id = 0

    # Se inicializa el archivo COCO JSON con el formato de las anotaciones
    coco_format = {
        "info": {},
        "licenses": [],
        "images": [],
        "categories": [{"id": value, "name": key, "supercategory": key} for key, value in category_ids.items()],
        "annotations": [],
    }

    # Se crean las secciones para las imágenes y anotaciones
    coco_format["images"], coco_format["annotations"], annotation_cnt = images_annotations_info(mask_path)

    # Por último se guarda el archivo
    with open(dest_json, "w") as outfile:
        json.dump(coco_format, outfile, sort_keys=True, indent=4)

    print("Se han creado %d anotaciones para las imágenes de la carpeta: %s" % (annotation_cnt, mask_path))


In [ ]:
from pathlib import Path

def batch_process_masks(root_dir, output_json_dir):
    root_dir = Path(root_dir)
    output_json_dir = Path(output_json_dir)
    output_json_dir.mkdir(parents=True, exist_ok=True)

    # Buscar todas las carpetas que contienen MASK.nii como nombre
    mask_dirs = [d for d in root_dir.rglob('*MASK.nii') if d.is_dir()]

    print(f"Se encontraron {len(mask_dirs)} carpetas con máscaras.")

    for mask_dir in tqdm(mask_dirs, desc="Procesando carpetas de máscara"):
        # Ruta relativa para mantener estructura
        relative_path = mask_dir.relative_to(root_dir)
        # Ruta destino del JSON replicando estructura, cambiando nombre de carpeta a archivo .json
        json_output_path = output_json_dir / relative_path.with_suffix('.json')

        # Asegurarse que la carpeta de destino existe
        json_output_path.parent.mkdir(parents=True, exist_ok=True)

        process_masks(str(mask_dir), str(json_output_path))

In [ ]:
batch_process_masks(
    '/content/drive/MyDrive/TFM/MSLesSeg-Dataset/output_trainMasks',
    '/content/drive/MyDrive/TFM/MSLesSeg-Dataset/annotations_trainMasks'
)

Se encontraron 92 carpetas con máscaras.


Procesando carpetas de máscara:   0%|          | 0/92 [00:00<?, ?it/s]

['/content/drive/MyDrive/TFM/MSLesSeg-Dataset/output_train/P1/T3/P1_T3_MASK.nii/slice_0.png', '/content/drive/MyDrive/TFM/MSLesSeg-Dataset/output_train/P1/T3/P1_T3_MASK.nii/slice_1.png', '/content/drive/MyDrive/TFM/MSLesSeg-Dataset/output_train/P1/T3/P1_T3_MASK.nii/slice_2.png', '/content/drive/MyDrive/TFM/MSLesSeg-Dataset/output_train/P1/T3/P1_T3_MASK.nii/slice_3.png', '/content/drive/MyDrive/TFM/MSLesSeg-Dataset/output_train/P1/T3/P1_T3_MASK.nii/slice_4.png', '/content/drive/MyDrive/TFM/MSLesSeg-Dataset/output_train/P1/T3/P1_T3_MASK.nii/slice_5.png', '/content/drive/MyDrive/TFM/MSLesSeg-Dataset/output_train/P1/T3/P1_T3_MASK.nii/slice_6.png', '/content/drive/MyDrive/TFM/MSLesSeg-Dataset/output_train/P1/T3/P1_T3_MASK.nii/slice_7.png', '/content/drive/MyDrive/TFM/MSLesSeg-Dataset/output_train/P1/T3/P1_T3_MASK.nii/slice_8.png', '/content/drive/MyDrive/TFM/MSLesSeg-Dataset/output_train/P1/T3/P1_T3_MASK.nii/slice_9.png', '/content/drive/MyDrive/TFM/MSLesSeg-Dataset/output_train/P1/T3/P1_T3


Procesando carpetas de máscara:   1%|          | 1/92 [00:03<04:49,  3.18s/it]

Se han creado 139 anotaciones para las imágenes de la carpeta: /content/drive/MyDrive/TFM/MSLesSeg-Dataset/output_train/P1/T3/P1_T3_MASK.nii
['/content/drive/MyDrive/TFM/MSLesSeg-Dataset/output_train/P1/T1/P1_T1_MASK.nii/slice_0.png', '/content/drive/MyDrive/TFM/MSLesSeg-Dataset/output_train/P1/T1/P1_T1_MASK.nii/slice_1.png', '/content/drive/MyDrive/TFM/MSLesSeg-Dataset/output_train/P1/T1/P1_T1_MASK.nii/slice_2.png', '/content/drive/MyDrive/TFM/MSLesSeg-Dataset/output_train/P1/T1/P1_T1_MASK.nii/slice_3.png', '/content/drive/MyDrive/TFM/MSLesSeg-Dataset/output_train/P1/T1/P1_T1_MASK.nii/slice_4.png', '/content/drive/MyDrive/TFM/MSLesSeg-Dataset/output_train/P1/T1/P1_T1_MASK.nii/slice_5.png', '/content/drive/MyDrive/TFM/MSLesSeg-Dataset/output_train/P1/T1/P1_T1_MASK.nii/slice_6.png', '/content/drive/MyDrive/TFM/MSLesSeg-Dataset/output_train/P1/T1/P1_T1_MASK.nii/slice_7.png', '/content/drive/MyDrive/TFM/MSLesSeg-Dataset/output_train/P1/T1/P1_T1_MASK.nii/slice_8.png', '/content/drive/MyDri

...

Se han creado 73 anotaciones para las imágenes de la carpeta: /content/drive/MyDrive/TFM/MSLesSeg-Dataset/output_train/P28/T2/P28_T2_MASK.nii
['/content/drive/MyDrive/TFM/MSLesSeg-Dataset/output_train/P28/T1/P28_T1_MASK.nii/slice_0.png', '/content/drive/MyDrive/TFM/MSLesSeg-Dataset/output_train/P28/T1/P28_T1_MASK.nii/slice_1.png', '/content/drive/MyDrive/TFM/MSLesSeg-Dataset/output_train/P28/T1/P28_T1_MASK.nii/slice_2.png', '/content/drive/MyDrive/TFM/MSLesSeg-Dataset/output_train/P28/T1/P28_T1_MASK.nii/slice_3.png', '/content/drive/MyDrive/TFM/MSLesSeg-Dataset/output_train/P28/T1/P28_T1_MASK.nii/slice_4.png', '/content/drive/MyDrive/TFM/MSLesSeg-Dataset/output_train/P28/T1/P28_T1_MASK.nii/slice_5.png', '/content/drive/MyDrive/TFM/MSLesSeg-Dataset/output_train/P28/T1/P28_T1_MASK.nii/slice_6.png', '/content/drive/MyDrive/TFM/MSLesSeg-Dataset/output_train/P28/T1/P28_T1_MASK.nii/slice_7.png', '/content/drive/MyDrive/TFM/MSLesSeg-Dataset/output_train/P28/T1/P28_T1_MASK.nii/slice_8.png', '/


Procesando carpetas de máscara:  99%|█████████▉| 91/92 [04:30<00:03,  3.27s/it]

Se han creado 35 anotaciones para las imágenes de la carpeta: /content/drive/MyDrive/TFM/MSLesSeg-Dataset/output_train/P28/T1/P28_T1_MASK.nii
['/content/drive/MyDrive/TFM/MSLesSeg-Dataset/output_train/P21/T1/P21_T1_MASK.nii/slice_0.png', '/content/drive/MyDrive/TFM/MSLesSeg-Dataset/output_train/P21/T1/P21_T1_MASK.nii/slice_1.png', '/content/drive/MyDrive/TFM/MSLesSeg-Dataset/output_train/P21/T1/P21_T1_MASK.nii/slice_2.png', '/content/drive/MyDrive/TFM/MSLesSeg-Dataset/output_train/P21/T1/P21_T1_MASK.nii/slice_3.png', '/content/drive/MyDrive/TFM/MSLesSeg-Dataset/output_train/P21/T1/P21_T1_MASK.nii/slice_4.png', '/content/drive/MyDrive/TFM/MSLesSeg-Dataset/output_train/P21/T1/P21_T1_MASK.nii/slice_5.png', '/content/drive/MyDrive/TFM/MSLesSeg-Dataset/output_train/P21/T1/P21_T1_MASK.nii/slice_6.png', '/content/drive/MyDrive/TFM/MSLesSeg-Dataset/output_train/P21/T1/P21_T1_MASK.nii/slice_7.png', '/content/drive/MyDrive/TFM/MSLesSeg-Dataset/output_train/P21/T1/P21_T1_MASK.nii/slice_8.png', '/


Procesando carpetas de máscara: 100%|██████████| 92/92 [04:33<00:00,  2.97s/it]

Se han creado 117 anotaciones para las imágenes de la carpeta: /content/drive/MyDrive/TFM/MSLesSeg-Dataset/output_train/P21/T1/P21_T1_MASK.nii


In [ ]:
process_masks(train_mask_path, train_json_path)
#process_masks(val_mask_path, val_json_path)

['/content/drive/MyDrive/TFM/MSLesSeg-Dataset/output_train/P1/T1/P1_T1_MASK.nii/slice_0.png', '/content/drive/MyDrive/TFM/MSLesSeg-Dataset/output_train/P1/T1/P1_T1_MASK.nii/slice_1.png', '/content/drive/MyDrive/TFM/MSLesSeg-Dataset/output_train/P1/T1/P1_T1_MASK.nii/slice_2.png', '/content/drive/MyDrive/TFM/MSLesSeg-Dataset/output_train/P1/T1/P1_T1_MASK.nii/slice_3.png', '/content/drive/MyDrive/TFM/MSLesSeg-Dataset/output_train/P1/T1/P1_T1_MASK.nii/slice_4.png', '/content/drive/MyDrive/TFM/MSLesSeg-Dataset/output_train/P1/T1/P1_T1_MASK.nii/slice_5.png', '/content/drive/MyDrive/TFM/MSLesSeg-Dataset/output_train/P1/T1/P1_T1_MASK.nii/slice_6.png', '/content/drive/MyDrive/TFM/MSLesSeg-Dataset/output_train/P1/T1/P1_T1_MASK.nii/slice_7.png', '/content/drive/MyDrive/TFM/MSLesSeg-Dataset/output_train/P1/T1/P1_T1_MASK.nii/slice_8.png', '/content/drive/MyDrive/TFM/MSLesSeg-Dataset/output_train/P1/T1/P1_T1_MASK.nii/slice_9.png', '/content/drive/MyDrive/TFM/MSLesSeg-Dataset/output_train/P1/T1/P1_T1

Procesando Lesion: 100%|██████████| 182/182 [00:01<00:00, 114.05imagen/s]


Se han creado 215 anotaciones para las imágenes de la carpeta: /content/drive/MyDrive/TFM/MSLesSeg-Dataset/output_train/P1/T1/P1_T1_MASK.nii
[]


Procesando Lesion: 0imagen [00:00, ?imagen/s]

Se han creado 0 anotaciones para las imágenes de la carpeta: /content/drive/MyDrive/TFM/MSLesSeg-Dataset/output_test/P54/T54/P54_T54_MASK.nii


In [ ]:
import cv2
import numpy as np
from pathlib import Path
from tqdm import tqdm
import json
import uuid

def generate_annotation(mask, image_id, annotation_id, category_id=1):
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    annotations = []

    for contour in contours:
        if cv2.contourArea(contour) < 5:
            continue  # Ignora contornos pequeños
        segmentation = contour.flatten().tolist()
        x, y, w, h = cv2.boundingRect(contour)
        area = float(cv2.contourArea(contour))

        annotation = {
            "id": annotation_id,
            "image_id": image_id,
            "category_id": category_id,
            "segmentation": [segmentation],
            "bbox": [x, y, w, h],
            "area": area,
            "iscrowd": 0
        }
        annotations.append(annotation)
        annotation_id += 1

    return annotations, annotation_id

def batch_masks_to_single_json(mask_dir, output_json_path):
    mask_dir = Path(mask_dir)
    mask_files = sorted(mask_dir.glob("*.png"))

    coco_output = {
        "images": [],
        "annotations": [],
        "categories": [
            {"id": 1, "name": "MS lesion"}
        ]
    }

    image_id = 1
    annotation_id = 1

    for mask_path in tqdm(mask_files, desc="Procesando máscaras"):
        mask = cv2.imread(str(mask_path), cv2.IMREAD_GRAYSCALE)
        height, width = mask.shape

        # Agregar entrada de imagen
        coco_output["images"].append({
            "id": image_id,
            "file_name": mask_path.name,
            "width": width,
            "height": height
        })

        # Generar anotaciones para esta máscara
        anns, annotation_id = generate_annotation(mask, image_id, annotation_id)
        coco_output["annotations"].extend(anns)

        image_id += 1

    # Guardar JSON
    with open(output_json_path, 'w') as f:
        json.dump(coco_output, f, indent=2)

    print(f"Anotaciones guardadas en: {output_json_path}")

In [ ]:
batch_masks_to_single_json(
    mask_dir="/content/drive/MyDrive/TFM/MSLesSeg-Dataset/RGBkCV/1CrossVal/trainMasks",
    output_json_path="/content/drive/MyDrive/TFM/MSLesSeg-Dataset/RGBkCV/1CrossVal/trainImages/trainMascaras-MSLesSeg.json"
)
batch_masks_to_single_json(
    mask_dir="/content/drive/MyDrive/TFM/MSLesSeg-Dataset/RGBkCV/2CrossVal/trainMasks",
    output_json_path="/content/drive/MyDrive/TFM/MSLesSeg-Dataset/RGBkCV/2CrossVal/trainImages/trainMascaras-MSLesSeg.json"
)
batch_masks_to_single_json(
    mask_dir="/content/drive/MyDrive/TFM/MSLesSeg-Dataset/RGBkCV/3CrossVal/trainMasks",
    output_json_path="/content/drive/MyDrive/TFM/MSLesSeg-Dataset/RGBkCV/3CrossVal/trainImages/trainMascaras-MSLesSeg.json"
)
batch_masks_to_single_json(
    mask_dir="/content/drive/MyDrive/TFM/MSLesSeg-Dataset/RGBkCV/4CrossVal/trainMasks",
    output_json_path="/content/drive/MyDrive/TFM/MSLesSeg-Dataset/RGBkCV/4CrossVal/trainImages/trainMascaras-MSLesSeg.json"
)
batch_masks_to_single_json(
    mask_dir="/content/drive/MyDrive/TFM/MSLesSeg-Dataset/RGBkCV/5CrossVal/trainMasks",
    output_json_path="/content/drive/MyDrive/TFM/MSLesSeg-Dataset/RGBkCV/5CrossVal/trainImages/trainMascaras-MSLesSeg.json"
)

Procesando máscaras: 100%|██████████| 13286/13286 [06:49<00:00, 32.45it/s] 


Anotaciones guardadas en: /content/drive/MyDrive/TFM/MSLesSeg-Dataset/RGBkCV/1CrossVal/trainImages/trainMascaras-MSLesSeg.json


Procesando máscaras: 100%|██████████| 13286/13286 [06:10<00:00, 35.88it/s] 


Anotaciones guardadas en: /content/drive/MyDrive/TFM/MSLesSeg-Dataset/RGBkCV/2CrossVal/trainImages/trainMascaras-MSLesSeg.json


Procesando máscaras: 100%|██████████| 13286/13286 [06:06<00:00, 36.26it/s] 


Anotaciones guardadas en: /content/drive/MyDrive/TFM/MSLesSeg-Dataset/RGBkCV/3CrossVal/trainImages/trainMascaras-MSLesSeg.json


Procesando máscaras: 100%|██████████| 13286/13286 [12:17<00:00, 18.00it/s] 


Anotaciones guardadas en: /content/drive/MyDrive/TFM/MSLesSeg-Dataset/RGBkCV/4CrossVal/trainImages/trainMascaras-MSLesSeg.json


Procesando máscaras:   1%|          | 77/7287 [02:54<4:32:32,  2.27s/it] 


KeyboardInterrupt: 

In [ ]:
batch_masks_to_single_json(
    mask_dir="/content/drive/MyDrive/TFM/MSLesSeg-Dataset/RGBkCV/1CrossVal/valMasks",
    output_json_path="/content/drive/MyDrive/TFM/MSLesSeg-Dataset/RGBkCV/1CrossVal/valImages/valMascaras-MSLesSeg.json"
)
batch_masks_to_single_json(
    mask_dir="/content/drive/MyDrive/TFM/MSLesSeg-Dataset/RGBkCV/2CrossVal/valMasks",
    output_json_path="/content/drive/MyDrive/TFM/MSLesSeg-Dataset/RGBkCV/2CrossVal/valImages/valMascaras-MSLesSeg.json"
)
#batch_masks_to_single_json(
#    mask_dir="/content/drive/MyDrive/TFM/MSLesSeg-Dataset/RGBkCV/3CrossVal/valMasks",
#    output_json_path="/content/drive/MyDrive/TFM/MSLesSeg-Dataset/RGBkCV/3CrossVal/valImages/valMascaras-MSLesSeg.json"
#)
#batch_masks_to_single_json(
#    mask_dir="/content/drive/MyDrive/TFM/MSLesSeg-Dataset/RGBkCV/4CrossVal/valMasks",
#    output_json_path="/content/drive/MyDrive/TFM/MSLesSeg-Dataset/RGBkCV/4CrossVal/valImages/valMascaras-MSLesSeg.json"
#)
#batch_masks_to_single_json(
#    mask_dir="/content/drive/MyDrive/TFM/MSLesSeg-Dataset/RGBkCV/5CrossVal/valMasks",
#    output_json_path="/content/drive/MyDrive/TFM/MSLesSeg-Dataset/RGBkCV/5CrossVal/valImages/valMascaras-MSLesSeg.json"
#)

Procesando máscaras: 100%|██████████| 3458/3458 [00:25<00:00, 134.96it/s]


Anotaciones guardadas en: /content/drive/MyDrive/TFM/MSLesSeg-Dataset/RGBkCV/1CrossVal/valImages/valMascaras-MSLesSeg.json


Procesando máscaras: 100%|██████████| 3458/3458 [01:25<00:00, 40.55it/s] 


Anotaciones guardadas en: /content/drive/MyDrive/TFM/MSLesSeg-Dataset/RGBkCV/2CrossVal/valImages/valMascaras-MSLesSeg.json


In [ ]:
import shutil
from pathlib import Path
from tqdm import tqdm

def flatten_combined_images(input_root, output_dir):
    input_root = Path(input_root)
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    png_files = sorted(input_root.rglob("slice_*.png"))

    print(f"Se encontraron {len(png_files)} imágenes combinadas.")

    for png_path in tqdm(png_files, desc="Copiando imágenes"):
        # Extraer partes del path: /train/P1/T1/slice_0.png -> P1_T1_slice_0.png
        parts = png_path.parts
        try:
            idx = parts.index("train") if "train" in parts else parts.index("test")
            patient = parts[idx + 1]
            scan_type = parts[idx + 2]
        except (ValueError, IndexError):
            print(f"No se pudo procesar ruta: {png_path}")
            continue

        new_name = f"{patient}_{scan_type}_{png_path.name}"
        dest_path = output_dir / new_name
        #print(dest_path)
        shutil.copy(png_path, dest_path)

    print(f"Imágenes copiadas en: {output_dir}")


In [ ]:
flatten_combined_images(
    input_root="/content/drive/MyDrive/TFM/MSLesSeg-Dataset/combined_rgb/train",
    output_dir="/content/drive/MyDrive/TFM/MSLesSeg-Dataset/RGBtrainImages"
)


Se encontraron 16744 imágenes combinadas.


Copiando imágenes: 100%|██████████| 16744/16744 [1:00:45<00:00,  4.59it/s]

Imágenes copiadas en: /content/drive/MyDrive/TFM/MSLesSeg-Dataset/RGBtrainImages


In [ ]:
from pathlib import Path

def verificar_coherencia_imgs_y_masks(rgb_root, mask_root):
    rgb_root = Path(rgb_root)
    mask_root = Path(mask_root)

    inconsistencias = 0

    print("📊 Verificando coherencia entre imágenes RGB y máscaras...\n")

    for patient_dir in sorted(rgb_root.glob("P*")):
        for scan_dir in sorted(patient_dir.glob("T*")):
            key = f"{patient_dir.name}_{scan_dir.name}"  # Ej: P1_T1

            # Contar imágenes RGB
            rgb_images = list(scan_dir.glob("slice_*.png"))
            n_rgb = len(rgb_images)

            # Buscar carpeta de máscaras
            mask_dir = mask_root / patient_dir.name / scan_dir.name / f"{key}_MASK.nii"
            if not mask_dir.exists():
                print(f"❌ No existe la carpeta de máscaras: {mask_dir}")
                inconsistencias += 1
                continue

            # Contar máscaras
            mask_images = list(mask_dir.glob("*.png"))
            n_masks = len(mask_images)

            # Comparar
            if n_rgb != n_masks:
                print(f"⚠️ Diferencia en {key}: {n_rgb} imágenes RGB vs {n_masks} máscaras")
                inconsistencias += 1

    if inconsistencias == 0:
        print("✅ Todas las imágenes y máscaras coinciden en número.")
    else:
        print(f"\n🔍 Se encontraron {inconsistencias} casos con discrepancias.")

In [ ]:
verificar_coherencia_imgs_y_masks(
    rgb_root="/content/drive/MyDrive/TFM/MSLesSeg-Dataset/combined_rgb/train",
    mask_root="/content/drive/MyDrive/TFM/MSLesSeg-Dataset/output_train"
)

📊 Verificando coherencia entre imágenes RGB y máscaras...

❌ No existe la carpeta de máscaras: /content/drive/MyDrive/TFM/MSLesSeg-Dataset/output_train/P30/T1/P30_T1_MASK.nii

🔍 Se encontraron 1 casos con discrepancias.


In [ ]:
from pathlib import Path
import shutil

def mover_imagenes_paciente(base_dir, paciente_scan, origen="trainImages", destino="testImages"):
    origen_dir = Path(base_dir) / origen
    destino_dir = Path(base_dir) / destino
    destino_dir.mkdir(parents=True, exist_ok=True)

    archivos = list(origen_dir.glob(f"{paciente_scan}_*.png"))

    for archivo in archivos:
        shutil.move(str(archivo), destino_dir / archivo.name)

    print(f"{len(archivos)} imágenes movidas de {origen} a {destino} para {paciente_scan}")

In [ ]:
mover_imagenes_paciente(
    base_dir="/content/drive/MyDrive/TFM/MSLesSeg-Dataset",
    paciente_scan="P30_T1"
)

182 imágenes movidas de trainImages a testImages para P30_T1


In [ ]:
import random
import shutil
from math import floor
from pathlib import Path

def dividir_y_mover_val_simple(
    train_images_dir, train_masks_dir,
    val_images_dir, val_masks_dir,
    seed=42, ratio_train=0.8
):
    train_images_dir = Path(train_images_dir)
    train_masks_dir = Path(train_masks_dir)
    val_images_dir = Path(val_images_dir)
    val_masks_dir = Path(val_masks_dir)

    val_images_dir.mkdir(parents=True, exist_ok=True)
    val_masks_dir.mkdir(parents=True, exist_ok=True)

    # Obtener todos los IDs de escaneos únicos, por ejemplo: P12_T2
    all_scan_ids = set(f.name.split('_slice')[0] for f in train_images_dir.glob("*.png"))
    all_scan_ids = sorted(all_scan_ids)

    total = len(all_scan_ids)
    num_train = floor(total * ratio_train)

    random.seed(seed)
    random.shuffle(all_scan_ids)

    train_scans = all_scan_ids[:num_train]
    val_scans = all_scan_ids[num_train:]

    print(f"📊 Total escaneos únicos: {total}")
    print(f"✅ Escaneos de entrenamiento: {len(train_scans)}")
    print(f"✅ Escaneos de validación: {len(val_scans)}")

    # Mover imágenes y máscaras para validación
    total_imgs, total_masks = 0, 0

    for scan_id in val_scans:
        imgs = list(train_images_dir.glob(f"{scan_id}_slice_*.png"))
        masks = list(train_masks_dir.glob(f"{scan_id}_slice_*.png"))

        for img in imgs:
            shutil.move(str(img), val_images_dir / img.name)
        for mask in masks:
            shutil.move(str(mask), val_masks_dir / mask.name)

        total_imgs += len(imgs)
        total_masks += len(masks)
        print(f"📁 {scan_id}: {len(imgs)} imágenes, {len(masks)} máscaras movidas.")

    print(f"\n✅ Total imágenes movidas: {total_imgs}")
    print(f"✅ Total máscaras movidas: {total_masks}")

    return train_scans, val_scans


In [ ]:
dividir_y_mover_val_simple(
    train_images_dir="/content/drive/MyDrive/TFM/MSLesSeg-Dataset/trainImages",
    train_masks_dir="/content/drive/MyDrive/TFM/MSLesSeg-Dataset/output_trainMasks",
    val_images_dir="/content/drive/MyDrive/TFM/MSLesSeg-Dataset/valImages",
    val_masks_dir="/content/drive/MyDrive/TFM/MSLesSeg-Dataset/valMasks"
)

📊 Total escaneos únicos: 92
✅ Escaneos de entrenamiento: 73
✅ Escaneos de validación: 19
📁 P53_T1: 182 imágenes, 182 máscaras movidas.
📁 P46_T1: 182 imágenes, 182 máscaras movidas.
📁 P22_T1: 182 imágenes, 182 máscaras movidas.
📁 P20_T3: 182 imágenes, 182 máscaras movidas.
📁 P6_T1: 182 imágenes, 182 máscaras movidas.
📁 P9_T1: 182 imágenes, 182 máscaras movidas.
📁 P12_T1: 182 imágenes, 182 máscaras movidas.
📁 P3_T1: 182 imágenes, 182 máscaras movidas.
📁 P51_T2: 182 imágenes, 182 máscaras movidas.
📁 P14_T2: 182 imágenes, 182 máscaras movidas.
📁 P4_T1: 182 imágenes, 182 máscaras movidas.
📁 P14_T4: 182 imágenes, 182 máscaras movidas.
📁 P18_T1: 182 imágenes, 182 máscaras movidas.
📁 P21_T1: 182 imágenes, 182 máscaras movidas.
📁 P23_T1: 182 imágenes, 182 máscaras movidas.
📁 P27_T1: 182 imágenes, 182 máscaras movidas.
📁 P11_T2: 182 imágenes, 182 máscaras movidas.
📁 P15_T1: 182 imágenes, 182 máscaras movidas.
📁 P5_T2: 182 imágenes, 182 máscaras movidas.

✅ Total imágenes movidas: 3458
✅ Total má

(['P42_T1',
  'P14_T1',
  'P31_T1',
  'P10_T2',
  'P47_T1',
  'P41_T1',
  'P16_T1',
  'P20_T2',
  'P2_T3',
  'P24_T1',
  'P33_T2',
  'P35_T1',
  'P49_T2',
  'P19_T2',
  'P43_T1',
  'P49_T1',
  'P32_T1',
  'P50_T1',
  'P4_T2',
  'P36_T1',
  'P37_T1',
  'P8_T1',
  'P52_T2',
  'P51_T1',
  'P25_T1',
  'P13_T1',
  'P28_T2',
  'P50_T2',
  'P22_T2',
  'P31_T2',
  'P52_T1',
  'P2_T4',
  'P2_T2',
  'P9_T2',
  'P40_T1',
  'P53_T2',
  'P14_T3',
  'P28_T1',
  'P1_T2',
  'P2_T1',
  'P19_T1',
  'P48_T1',
  'P3_T2',
  'P45_T1',
  'P12_T4',
  'P26_T1',
  'P9_T3',
  'P33_T1',
  'P11_T1',
  'P17_T1',
  'P29_T1',
  'P38_T1',
  'P1_T1',
  'P3_T3',
  'P1_T3',
  'P12_T2',
  'P12_T3',
  'P44_T1',
  'P34_T1',
  'P6_T3',
  'P13_T2',
  'P7_T1',
  'P19_T4',
  'P5_T1',
  'P19_T3',
  'P10_T1',
  'P8_T2',
  'P3_T4',
  'P7_T2',
  'P39_T1',
  'P6_T2',
  'P20_T1',
  'P4_T3'],
 ['P53_T1',
  'P46_T1',
  'P22_T1',
  'P20_T3',
  'P6_T1',
  'P9_T1',
  'P12_T1',
  'P3_T1',
  'P51_T2',
  'P14_T2',
  'P4_T1',
  'P14_T4',
  'P

In [ ]:
from pathlib import Path
import re

def corregir_nombres_test_images(image_dir):
    image_dir = Path(image_dir)
    pattern = re.compile(r"(P\d+_slice_\d+)\.png_slice_\d+\.png")

    renamed = 0

    for file in image_dir.glob("*.png*"):
        match = pattern.match(file.name)
        if match:
            nuevo_nombre = match.group(1) + ".png"
            nuevo_path = image_dir / nuevo_nombre
            file.rename(nuevo_path)
            print(f"✅ Renombrado: {file.name} → {nuevo_nombre}")
            renamed += 1

    print(f"\n🔁 Total de archivos renombrados: {renamed}")

# 🧪 Ejemplo de uso
corregir_nombres_test_images("/content/drive/MyDrive/TFM/MSLesSeg-Dataset/testImages")

✅ Renombrado: P70_slice_2.png_slice_2.png → P70_slice_2.png
✅ Renombrado: P70_slice_20.png_slice_20.png → P70_slice_20.png
✅ Renombrado: P70_slice_21.png_slice_21.png → P70_slice_21.png
✅ Renombrado: P70_slice_22.png_slice_22.png → P70_slice_22.png
✅ Renombrado: P70_slice_23.png_slice_23.png → P70_slice_23.png
✅ Renombrado: P70_slice_24.png_slice_24.png → P70_slice_24.png
✅ Renombrado: P70_slice_25.png_slice_25.png → P70_slice_25.png
✅ Renombrado: P70_slice_26.png_slice_26.png → P70_slice_26.png
✅ Renombrado: P70_slice_27.png_slice_27.png → P70_slice_27.png
✅ Renombrado: P70_slice_28.png_slice_28.png → P70_slice_28.png
✅ Renombrado: P70_slice_29.png_slice_29.png → P70_slice_29.png
✅ Renombrado: P70_slice_3.png_slice_3.png → P70_slice_3.png
✅ Renombrado: P70_slice_30.png_slice_30.png → P70_slice_30.png
✅ Renombrado: P70_slice_31.png_slice_31.png → P70_slice_31.png
✅ Renombrado: P70_slice_32.png_slice_32.png → P70_slice_32.png
✅ Renombrado: P70_slice_33.png_slice_33.png → P70_slice_33.pn

In [ ]:
import random
import shutil
from pathlib import Path
from math import ceil
from tqdm import tqdm

def k_fold_split_images_masks(
    images_dir, masks_dir, output_dir,
    k=5, seed=42, move_files=False
):
    images_dir = Path(images_dir)
    masks_dir = Path(masks_dir)
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    # Extraer escaneos únicos: P1_T1, P12_T2, etc.
    all_scan_ids = sorted(set(f.name.split('_slice')[0] for f in images_dir.glob("*.png")))
    random.seed(seed)
    random.shuffle(all_scan_ids)

    # Dividir escaneos en K partes
    fold_size = ceil(len(all_scan_ids) / k)
    folds = [all_scan_ids[i * fold_size: (i + 1) * fold_size] for i in range(k)]

    print(f"📦 Total escaneos únicos: {len(all_scan_ids)}")
    print(f"🔁 Dividido en {k} folds, de tamaño aproximado {fold_size} escaneos por fold.")

    # Elegir método de transferencia
    file_op = shutil.move if move_files else shutil.copy

    for fold_idx in range(k):
        val_scans = set(folds[fold_idx])
        train_scans = set(all_scan_ids) - val_scans

        print(f"\n📁 Fold {fold_idx + 1}")
        print(f"🔍 Escaneos de validación ({len(val_scans)}): {sorted(val_scans)}")

        fold_path = output_dir / f"{fold_idx+1}CrossVal"
        train_img_dir = fold_path / "trainImages"
        val_img_dir = fold_path / "valImages"
        train_mask_dir = fold_path / "trainMasks"
        val_mask_dir = fold_path / "valMasks"

        # Crear carpetas
        for d in [train_img_dir, val_img_dir, train_mask_dir, val_mask_dir]:
            d.mkdir(parents=True, exist_ok=True)

        def transfer_files(scan_ids, src_dir, dst_dir, desc=""):
            matched_files = []
            for scan_id in scan_ids:
                matched_files.extend(src_dir.glob(f"{scan_id}_slice_*.png"))
            for f in tqdm(matched_files, desc=desc, leave=False):
                file_op(f, dst_dir / f.name)
            return len(matched_files)

        train_imgs = transfer_files(train_scans, images_dir, train_img_dir, desc="📤 Copiando imágenes de entrenamiento")
        train_masks = transfer_files(train_scans, masks_dir, train_mask_dir, desc="📤 Copiando máscaras de entrenamiento")
        val_imgs = transfer_files(val_scans, images_dir, val_img_dir, desc="📥 Copiando imágenes de validación")
        val_masks = transfer_files(val_scans, masks_dir, val_mask_dir, desc="📥 Copiando máscaras de validación")

        print(f"   🏋️‍♂️ Train: {len(train_scans)} escaneos -> {train_imgs} imágenes, {train_masks} máscaras")
        print(f"   ✅ Val: {len(val_scans)} escaneos -> {val_imgs} imágenes, {val_masks} máscaras")

    print("\n✅ K-Fold división completada.")

In [ ]:
k_fold_split_images_masks(
    "/content/drive/MyDrive/TFM/MSLesSeg-Dataset/RGBtrainImages",
    "/content/drive/MyDrive/TFM/MSLesSeg-Dataset/output_trainMasks",
    "/content/drive/MyDrive/TFM/MSLesSeg-Dataset/RGBkCV",
    k=5, seed=42, move_files=False)

📦 Total escaneos únicos: 92
🔁 Dividido en 5 folds, de tamaño aproximado 19 escaneos por fold.

📁 Fold 1
🔍 Escaneos de validación (19): ['P10_T2', 'P14_T1', 'P16_T1', 'P19_T2', 'P20_T2', 'P24_T1', 'P2_T3', 'P31_T1', 'P32_T1', 'P33_T2', 'P35_T1', 'P41_T1', 'P42_T1', 'P43_T1', 'P47_T1', 'P49_T1', 'P49_T2', 'P4_T2', 'P50_T1']


   🏋️‍♂️ Train: 73 escaneos -> 13286 imágenes, 13286 máscaras
   ✅ Val: 19 escaneos -> 3458 imágenes, 3458 máscaras

📁 Fold 2
🔍 Escaneos de validación (19): ['P13_T1', 'P14_T3', 'P22_T2', 'P25_T1', 'P28_T1', 'P28_T2', 'P2_T2', 'P2_T4', 'P31_T2', 'P36_T1', 'P37_T1', 'P40_T1', 'P50_T2', 'P51_T1', 'P52_T1', 'P52_T2', 'P53_T2', 'P8_T1', 'P9_T2']


   🏋️‍♂️ Train: 73 escaneos -> 13286 imágenes, 13286 máscaras
   ✅ Val: 19 escaneos -> 3458 imágenes, 3458 máscaras

📁 Fold 3
🔍 Escaneos de validación (19): ['P11_T1', 'P12_T2', 'P12_T3', 'P12_T4', 'P17_T1', 'P19_T1', 'P1_T1', 'P1_T2', 'P1_T3', 'P26_T1', 'P29_T1', 'P2_T1', 'P33_T1', 'P38_T1', 'P3_T2', 'P3_T3', 'P45_T1', 'P48_T1', 'P9_T3']


   🏋️‍♂️ Train: 73 escaneos -> 13286 imágenes, 13286 máscaras
   ✅ Val: 19 escaneos -> 3458 imágenes, 3458 máscaras

📁 Fold 4
🔍 Escaneos de validación (19): ['P10_T1', 'P13_T2', 'P19_T3', 'P19_T4', 'P20_T1', 'P22_T1', 'P34_T1', 'P39_T1', 'P3_T4', 'P44_T1', 'P46_T1', 'P4_T3', 'P53_T1', 'P5_T1', 'P6_T2', 'P6_T3', 'P7_T1', 'P7_T2', 'P8_T2']


   🏋️‍♂️ Train: 73 escaneos -> 13286 imágenes, 13286 máscaras
   ✅ Val: 19 escaneos -> 3458 imágenes, 3458 máscaras

📁 Fold 5
🔍 Escaneos de validación (16): ['P11_T2', 'P12_T1', 'P14_T2', 'P14_T4', 'P15_T1', 'P18_T1', 'P20_T3', 'P21_T1', 'P23_T1', 'P27_T1', 'P3_T1', 'P4_T1', 'P51_T2', 'P5_T2', 'P6_T1', 'P9_T1']


   🏋️‍♂️ Train: 76 escaneos -> 13832 imágenes, 13832 máscaras
   ✅ Val: 16 escaneos -> 2912 imágenes, 2912 máscaras

✅ K-Fold división completada.


In [ ]:
from pathlib import Path; import shutil

for f in Path("/content/drive/MyDrive/TFM/MSLesSeg-Dataset/valMasks").glob("*.png"):
    shutil.copy(f, Path("/content/drive/MyDrive/TFM/MSLesSeg-Dataset/output_trainMasks") / f.name)